# 风险溢价

## 加载模块

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats
from scipy.stats import mstats
import statsmodels.api as sm

## 读入文件

In [2]:
df_merge = pd.read_csv('../data/stock-accounting.csv')
df_merge['date'] = pd.to_datetime(df_merge['date'])

## Fama-MacBeth 回归

剔除 `nan` 数据

_Warning: `size` 使用的是 `market-cap` 的自然对数_

_Warning: `investment` 使用的是相邻年份 `at` 的直接相除，与文章中定义相同，与文章表格中不同_

In [3]:
df_merge['size'] = df_merge['market-cap'].apply(np.log)

idx_fac = ['beta', 'size', 'book-to-market', 'profit', 'investment', 'past-return']
idx_par = ['const'] + idx_fac
df_fit = df_merge[['PERMNO', 'date', 'stg', 'RET-RF'] + idx_fac].copy()
df_fit['investment'] = df_fit['investment'].apply(np.exp)
# df_fit[idx_fac] = df_fit.groupby('PERMNO')[idx_fac].shift(1)
df_fit['RET-RF'] = df_fit.groupby('PERMNO')['RET-RF'].shift(-1)
df_fit = df_fit.replace([np.inf, -np.inf], np.nan)
df_fit = df_fit.dropna()
df_fit[idx_fac] = df_fit[idx_fac].apply(mstats.winsorize, limits=[0.01, 0.01])
df_fit.shape[0]

991910

In [4]:
time_series = df_fit['date'].unique()
time_series = np.sort(time_series)

idx_stg = ['Intro', 'Growth', 'Mature', 'Shakeout', 'Decline']

df_params = pd.DataFrame(columns=['stg', 'date'] + idx_par)
df_fit_info = pd.DataFrame(columns=['stg', 'date', 'R Squared', 'Observations'])

for stg in ['All'] + idx_stg:
    if stg == 'All':
        idx = True
    else:
        idx = (df_fit['stg'] == stg)

    for t in time_series[:-1]:
        df_date = df_fit[(idx) & (df_fit['date'] == t)].copy()
        x = df_date[idx_fac]
        z = (x - x.mean()) / x.std()
        z = z.clip(-3, 3)
        z = sm.add_constant(z)
        y = df_date['RET-RF']
        result = sm.OLS(y, z).fit()
        df_params.loc[len(df_params)] = [stg, t] + list(result.params)
        df_fit_info.loc[len(df_params)] = [stg, t, result.rsquared, result.nobs]

### EXHIBIT 8

先定义 significance

In [5]:
def significance_p(pvalue):
    if pvalue < 0.01:
        return '***'
    elif pvalue < 0.05:
        return '**'
    elif pvalue < 0.1:
        return '*'
    else:
        return ''

In [6]:
df_reg = pd.DataFrame(index=idx_par, columns=['All'] + idx_stg + ['F-Test'])
df_reg_t = pd.DataFrame(index=idx_par, columns=['All'] + idx_stg + ['F-Test'])
df_reg_s = pd.DataFrame(index=idx_par, columns=['All'] + idx_stg + ['F-Test'])
df_reg_info = pd.DataFrame(index=['R Squared', 'Observations'], columns = ['All'] + idx_stg + ['F-Test'])
for stg in ['All'] + idx_stg:
    df_par = df_params.loc[df_params['stg'] == stg, idx_par]
    df_reg[stg] = df_par.mean() * 100
    t = np.sqrt(407) * df_par.mean() / df_par.std()
    df_reg_t[stg] = t
    p = 2 * (1 - t.apply(abs).apply(stats.t.cdf, df=407 - 1))
    df_reg_s[stg] = p.apply(significance_p)
    df_reg_info.loc['R Squared', stg] = df_fit_info.loc[df_fit_info['stg'] == stg, 'R Squared'].mean()
    df_reg_info.loc['Observations', stg] = df_fit_info.loc[df_fit_info['stg'] == stg, 'Observations'].sum()
df_reg_t = df_reg_t.rename({p: p + '-t' for p in idx_par})
df_reg_s = df_reg_s.rename({p: p + '-s' for p in idx_par})
df_reg = pd.concat([df_reg, df_reg_t, df_reg_s, df_reg_info])
df_reg = df_reg.loc[['beta', 'beta-t', 'beta-s', 'size', 'size-t', 'size-s',
                 'book-to-market', 'book-to-market-t', 'book-to-market-s',
                 'profit', 'profit-t', 'profit-s', 'investment', 'investment-t', 'investment-s',
                 'past-return', 'past-return-t', 'past-return-s', 'const', 'const-t', 'const-s',
                 'R Squared', 'Observations']]

F-Test

In [7]:
for fac in idx_fac:
    group_coef = [100 * df_params.loc[df_params['stg'] == stg, fac] for stg in idx_stg]
    F, p = stats.f_oneway(*group_coef)
    df_reg.loc[fac, 'F-Test'] = F
    df_reg.loc[fac + '-s', 'F-Test'] = significance_p(p)

In [8]:
df_reg.to_csv('../exhibits/Exhibit8.csv')
pd.options.display.float_format = '{:.3f}'.format
df_reg

,All,Intro,Growth,Mature,Shakeout,Decline,F-Test
beta,0.109,0.125,0.076,0.160,0.164,0.282,0.416
beta-t,1.118,0.902,0.765,1.940,1.380,2.013,NaN
beta-s,,,,*,,**,
size,-0.208,-0.447,-0.137,-0.241,-0.284,-0.383,1.032
size-t,-2.340,-3.021,-1.637,-3.177,-2.837,-2.357,NaN
size-s,**,***,,***,***,**,
book-to-market,0.416,0.350,0.205,0.356,0.574,0.898,4.123
book-to-market-t,5.488,2.224,2.064,4.238,4.488,5.166,NaN
book-to-market-s,***,**,**,***,***,***,***
profit,0.301,0.248,0.135,0.180,0.203,0.086,0.293


## 多空投资组合：全样本估值

### 全样本估算超额收益率

In [9]:
df_coef_full = df_params.groupby('stg')[idx_par].mean()

df_shift = df_merge[['PERMNO', 'date', 'EXCHCD', 'stg', 'adj-prc',
                     'RET', 'RET-RF', 'RF', 'market-cap']].copy()
df_shift[['RET', 'RET-RF', 'RF']] = df_shift.groupby('PERMNO')[['RET', 'RET-RF', 'RF']].shift(-1)
# df_shift['market-cap'] = df_shift.groupby('PERMNO')['market-cap'].shift(1)
df_shift = df_shift.iloc[df_fit.index]

x_mean = df_fit.groupby('date')[idx_fac].transform('mean')
x_std = df_fit.groupby('date')[idx_fac].transform('std')
z_un = (df_fit[idx_fac] - x_mean) / x_std
z_un = z_un.clip(-3, 3)
z_un['const'] = 1
x_mean = df_fit.groupby(['date', 'stg'])[idx_fac].transform('mean')
x_std = df_fit.groupby(['date', 'stg'])[idx_fac].transform('std')
z_stg = (df_fit[idx_fac] - x_mean) / x_std
z_stg = z_stg.clip(-3, 3)
z_stg['const'] = 1

ret_un_full = (z_un[idx_par] * df_coef_full.loc['All', idx_par]).sum(axis=1)
ret_un_full.name = 'pred-RET-RF'
df_stg = df_fit[['stg']].replace('Unknown', 'All')
df_coef_stg_full = df_stg.join(df_coef_full, on='stg')
ret_stg_full = (z_stg[idx_par] * df_coef_stg_full[idx_par]).sum(axis=1)
ret_stg_full.name = 'pred-RET-RF'

### 计算不同组合内月平均超额收益率

定义函数，根据预测的超额收益率计算不同组的超额收益率

In [10]:
def share_weight(group):
    g = group.copy()
    weight = g['market-cap'] / g['market-cap'].sum()
    weight.fillna(0)
    weight.name = 'weight'
    return pd.concat([g['PERMNO'], weight], axis=1)
def exc_ret(group):
    g = group.copy()
    rf = g['RF'].mean()
    return (g['RET'] * g['weight']).sum() - rf

ret_group = ['low', '2', '3', '4', 'high']

def exc_calc(ret_exp):
    df_exc = pd.DataFrame(columns=['date'] + ret_group)
    ret = ret_exp.copy()
    for i, t in enumerate(time_series[1:-1]):
        idx = (df_shift['date'] == t)
        idx2 = idx & (df_shift['EXCHCD'] == 1)
        ret_date = ret[idx].copy()
        shift_date = df_shift[idx].copy()
        q = ret.loc[idx2].quantile([0., 0.2, 0.4, 0.6, 0.8, 1.])
        qcut = pd.cut(ret_date, q, right=True, labels=ret_group)
        qcut[ret_date <= q[0.2]] = 'low'
        qcut[ret_date > q[0.8]] = 'high'
        qcut.name = 'qcut'
        shift_date = pd.concat([shift_date, qcut], axis=1)
        weight = shift_date.groupby('qcut', observed=False)[['PERMNO', 'adj-prc', 'market-cap']].apply(share_weight).reset_index()
        shift_date = pd.merge(
            shift_date,
            weight[['PERMNO', 'weight']],
            on='PERMNO', how='left'
        )
        df_exc.loc[i, 'date'] = t
        df_exc.loc[i, ret_group] = shift_date.groupby('qcut', observed=False)[['adj-prc', 'RET', 'RET-RF', 'weight', 'RF']].apply(exc_ret)
    return df_exc

In [11]:
exc_un_full = exc_calc(ret_un_full)
group_un_full = exc_un_full[ret_group].mean() * 100
print(group_un_full)
print(group_un_full['high'] - group_un_full['low'])
exc_stg_full = exc_calc(ret_stg_full)
group_stg_full = exc_stg_full[ret_group].mean() * 100
print(group_stg_full)
print(group_stg_full['high'] - group_stg_full['low'])

low    0.667
2      0.851
3      1.024
4      1.010
high   1.152
dtype: object
0.4849853401954203
low    0.611
2      0.884
3      0.889
4      1.147
high   1.310
dtype: object
0.6997905430051373


## 多空投资组合：滚动窗口估值

### 滚动窗口计算拟合系数平均值

In [12]:
df_params_shift = pd.DataFrame(columns=df_params.columns)
df_params_shift[['stg', 'date']] = df_params[['stg', 'date']]
df_params_shift[idx_par] = df_params.groupby('stg')[idx_par].shift(1)
df_coef_expand = (df_params_shift
    .set_index('date')
    .groupby('stg')
    .expanding()[idx_par]
    .mean()
    .reset_index()
)

### 滚动窗口估算超额收益率

In [13]:
df_coef_expand = df_coef_expand.set_index(['stg', 'date'])
coef_date = df_fit[['date']].join(df_coef_expand.loc['All'], on='date')
ret_un_expand = (z_un[idx_par] * coef_date[idx_par]).sum(axis=1)
ret_un_expand.name = 'pred-RET-RF'

coef_stg_date = (df_fit[['stg', 'date']]
    .replace('Unknown', 'All')
    .join(df_coef_expand, on=['stg', 'date'])
)
ret_stg_expand = (z_stg[idx_par] * coef_stg_date[idx_par]).sum(axis=1)
ret_stg_expand.name = 'pred-RET-RF'

print(ret_un_expand)
print(ret_stg_expand)

23        -0.012
24        -0.011
25        -0.006
26         0.008
27         0.009
           ...  
1613606    0.007
1613607    0.006
1613608    0.007
1613609    0.006
1613610    0.005
Name: pred-RET-RF, Length: 991910, dtype: float64
23        -0.008
24        -0.007
25        -0.001
26         0.013
27         0.011
           ...  
1613606    0.008
1613607    0.008
1613608    0.008
1613609    0.008
1613610    0.007
Name: pred-RET-RF, Length: 991910, dtype: float64


用循环验证

In [14]:
df_coef_expand_2 = pd.DataFrame(index=df_fit.index, columns=['stg', 'date'] + idx_par)
df_coef_expand_2[['stg', 'date']] = df_fit[['stg', 'date']]
for t in time_series:
    pass

### 计算不同组合内月平均超额收益率

In [15]:
exc_un_expand = exc_calc(ret_un_expand)
group_un_expand = exc_un_expand[ret_group].mean() * 100
print(group_un_expand)
print(group_un_expand['high'] - group_un_expand['low'])
exc_stg_expand = exc_calc(ret_stg_expand)
group_stg_expand = exc_stg_expand[ret_group].mean() * 100
print(group_stg_expand)
print(group_stg_expand['high'] - group_stg_expand['low'])

low    0.618
2      0.811
3      1.081
4      1.015
high   1.214
dtype: object
0.5958809646309838
low    0.621
2      0.748
3      0.816
4      1.088
high   1.338
dtype: object
0.7170279976195163


## EXHIBIT 9